In [ ]:
include("_.jl")

In [ ]:
tr = SE["tree"]

isdir(tr)

In [ ]:
em = SE["example_md"]

he_ = [li for li in readlines(em) if li != ""]

In [ ]:
println("Renaming .md")

for (ro, di_, fi_) in walk_up(tr)

    for fi in fi_

        cl = Kwat.path.clean(fi)

        if fi != cl

            println(fi, " ==> ", cl)

            mv(joinpath(ro, fi), joinpath(ro, cl))

        end

    end

end

In [ ]:
println("Numbering 1._.md")

for (ro, di_, fi_) in walk_up(tr)

    for fi in fi_

        id, ti = split(fi, ".")

        if ti == "_"

            if id != "1"

                pa = joinpath(ro, fi)

                println("ID is not 1 (re-IDing): ", pa)

                mv(pa, joinpath(ro, string("1._.md")))

            end

        end

    end

end

In [ ]:
println("Numbering / and .md")

for (ro, di_, fi_) in walk_up(tr)

    go_ = Vector{String}()

    ba_ = Vector{String}()

    for na in [di_; fi_]

        if is_good_structure(na)

            push!(go_, na)

        else

            push!(ba_, na)

        end

    end

    sort!(go_; by = split_good_structure)

    n_go = length(go_)

    for (id, na) in enumerate([go_; ba_])

        pa1 = joinpath(ro, na)

        if id <= n_go

            pa2 = joinpath(ro, join([id; split_good_structure(na)[2:end]], "."))

        else

            pa2 = joinpath(ro, join([id, na], "."))

        end

        if pa1 != pa2

            println(splitdir(pa1)[2], " ==> ", splitdir(pa2)[2])

            mv(pa1, pa2)

        end

    end

end

In [ ]:
println("Checking .md content")

for (ro, di_, fi_) in walk_up(tr)

    for fi in fi_

        pa = joinpath(ro, fi)

        li_ = readlines(pa)

        if length(li_) == 0

            println("Is empty (populating): ", pa)

            cp(em, pa; force = true)

        elseif !all(he in li_ for he in he_)

            #error("Is missing at least 1 required header: ", pa)

        elseif !all(0 .< diff([findfirst(li_ .== he) for he in he_]))

            error("Has incorrectly ordered headers: ", pa)

        end

    end

end

In [ ]:
println("IDing .md")

ti_id = Dict{String,Vector{Int64}}()

for (ro, di_, fi_) in walk_up(tr)

    for fi in fi_

        id, ti = split(fi, ".")

        if ti == "_"

            if id != "1"

                error("ID is not 1: ", fi)

            end

            ti = split(splitdir(ro)[2], ".")[2]

        end

        pa = joinpath(ro, fi)

        if haskey(ti_id, ti)

            error("Duplicated: ", ti_id[ti], " and ", pa)

        else

            if occursin("  ", ti)
                println(ti)
            end

            ti_id[ti] = make_id(pa)

        end

    end

end